In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
import importlib

import training.preprocess as preprocess
import testing.tagger as tagger
import testing.syllabification as syllabification
import config
import testing.stemmer as stemmer

from pprint import pprint

## Import result data

In [44]:
fpath = "../../data/g2p/var/stem+rule/best/se_stem_gkn_n=7_B=18_fold_5.txt"

res_data = pd.read_csv(
    fpath,
    sep='\t',
    header=None,
    names=['word', 'syllables', 'prediction', 'mmc'],
    na_filter=False
)
res_data.head(5)

,word,syllables,prediction,mmc
0,a,a,a,0
1,a.ba-a.ba,a.ba-a.ba,a.ba-a.ba,0
2,a.bah,a.bah,a.bah,0
3,a.ba.ka,a.ba.ka,a.ba.ka,0
4,a.ba.ngan,a.ba.)*an,a.ba.)*an,0


## Wrong phoneme frequencies

In [45]:
wrong_phonemes = []
wrong_phonemes_dict = {}

for row in res_data.itertuples():
    if row.mmc == 0:
        wrong_phonemes.append("-")
    else:
        wp_string = ""
        c = 0
        for i in range(len(row.syllables)):
            if row.syllables[i] != row.prediction[i]:
                wp = f"{row.syllables[i]}/{row.prediction[i]}"
                wp_string += wp

                if wp not in wrong_phonemes_dict:
                    wrong_phonemes_dict[wp] = 0
                wrong_phonemes_dict[wp] += 1

                c += 1

                if c < row.mmc:
                    wp += " "

        wrong_phonemes.append(wp_string)

res_data_new = res_data.copy()
res_data_new["wrong_phonemes"] = wrong_phonemes

freq_data_list = []

for phonemes_str, freq in wrong_phonemes_dict.items():
    phonemes = phonemes_str.split("/")
    freq_data_list.append((phonemes[0], phonemes[1], freq))

freq_data = pd.DataFrame(freq_data_list, columns=["real_phoneme", "pred_phoneme", "frequency"]).sort_values("frequency", ascending=False).reset_index(drop=True)

freq_data

,real_phoneme,pred_phoneme,frequency
0,#,e,293
1,e,#,260
2,5,o,7
3,1,a,7
4,*,i,6
5,4,i,5
6,$,1,4
7,*,h,3
8,2,e,3
9,o,5,2


In [46]:
fpath = "../../data/g2p/var/stem+rule/best/wrong_freqs_fold_5.txt"

freq_data.to_csv(
    fpath,
    sep='\t',
    index=False,
    header=False
)

## Error source frequencies

In [37]:
wrong_types = []
wrong_types_dict = {"affix": 0, "root": 0, "both": 0}

st = stemmer.Stemmer()

for row in res_data.itertuples():
    if row.mmc == 0:
        wrong_types.append("-")
    else:
        prefix, root, d_suffix, i_suffix = st.getRoot(row.word)

        if prefix == '' and d_suffix == '' and i_suffix == '':
            wrong_types.append("root")
            wrong_types_dict["root"] += 1
            continue
        
        affix = False
        root = False

        for i in range(len(row.syllables)):
            if row.syllables[i] != row.prediction[i]:
                if i >= len(prefix) and i < (len(row.word) - len(d_suffix) - len(i_suffix)):
                    root = True
                else:
                    affix = True
            
        if affix and root:
            wrong_types.append("both")
            wrong_types_dict["both"] += 1
        elif affix:
            wrong_types.append("affix")
            wrong_types_dict["affix"] += 1
        else:
            wrong_types.append("root")
            wrong_types_dict["root"] += 1

res_data_new = res_data.copy()
res_data_new["wrong_types"] = wrong_types

pprint(wrong_types_dict)
print(f"Total word error: {sum(wrong_types_dict[x] for x in wrong_types_dict)}")

{'affix': 7, 'both': 0, 'root': 535}
Total word error: 542


## Save to csv

In [ ]:
fpath = "../../data/g2p/var/wrong_types_vanilla.txt"

res_data_new.to_csv(
    fpath,
    sep='\t', 
)